In [ ]:
import scipy.io as scio
import numpy as np    
import matplotlib.pyplot as plt
import sys
import os
import math
import cv2
from scipy.misc import imsave
import scipy.ndimage as ndimage
from unet import UNet
from loss import *
import torch
import torch.nn as nn
import heapq
from torch.autograd import Variable
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
%load_ext autoreload
%autoreload 2

In [18]:
value = 1
def get_info(filenames, ext, root):
    images = []
    for filename in filenames :
        filepath = os.path.join(root,filename)
        if ext == '.npy':
            image = np.load(filepath)
        elif ext == '.JPG' or ext == '.tif' or ext =='.png' or ext =='.tiff':
            image = ndimage.imread(filepath)
        images.append(image)
    return images

def get_data(directory,ext):
    from os import listdir
    from os.path import isfile, join
    
    root_path = ""
    filenames = [f for f in listdir(directory) if isfile(join(directory, f)) and f != '.DS_Store']
    filenames = sorted(filenames)
    return filenames, get_info(filenames, ext, directory)

In [ ]:
cwd = os.getcwd()
filenames_im, images = get_data(cwd+'\medical_images\\oct_images\\Train','.png')
filenames_lb, lbs = get_data(cwd+'\medical_images\\training_set','.png')
print(len(filenames_im))


In [ ]:
model = UNet(n_channels=1, n_classes=2).cuda()
#model = nn.DataParallel(model)
PATH = os.getcwd() + '\\models\\model_noise_75_1_8\\1it\\seg_module.model'
model.load_state_dict(torch.load(PATH))
model.eval()

In [ ]:
#from relaynet_pytorch.net_api.losses import CombinedLoss
loss_func=CombinedLoss()
total_loss = []

for j in range(len(filenames_im)):
    im = images[j].copy()
    w = np.ones([1,1,im.shape[0], im.shape[1]])
    label = lbs[j].reshape((1,im.shape[0], -1))
    im = im.reshape((1,1,im.shape[0], -1))    
    im = torch.from_numpy(im) 
    w = torch.from_numpy(w) 
    label = torch.from_numpy(label)

    y = Variable(label).cuda().float()
    X = Variable(im).cuda().float()
    w = Variable(w).cuda().float()
    output = model(X)
    output2 = output.data.cpu().numpy()
    output2 = np.squeeze(output2[0])
    output2 = np.argmax(output2,0)
    output2 = np.squeeze(output2)

    loss = loss_func(output.float(), y.float(), w.float(), num_classes = 2)
    total_loss.append(loss[2].item())
print(total_loss)

In [22]:
# sort the loss values
nlevel = 0.75
max_indexes = map(total_loss.index, heapq.nlargest(int(124*nlevel), total_loss))
max_index_k = list(max_indexes)
#print(len(max_index_k), max_index_k)
#for index_k in max_index_k:
    #print(total_loss[index_k])

In [ ]:
cwd = os.getcwd()
check_directory = cwd
filenames_lb, lbs = get_data(cwd+'\medical_images\\\\training_set','.png')
filenames_pred, pred = get_data(cwd+'\\medical_images\\prediction','.png')
filenames_gt, gts = get_data(cwd+'\medical_images\\odd_gt','.png')
gts = np.array(gts)
gts[gts >1] =1
print(len(filenames_lb))
print(len(filenames_pred))

In [ ]:
for j in range(len(filenames_im)):
    print(filenames_im[j])
    if j in max_index_k:
        im = images[j].copy()
        im = im*1.0/np.max(im)
        im = im.reshape((1,1,im.shape[0], -1))
        im = torch.from_numpy(im)
        X = Variable(im).cuda().float()
        out_map = []
        number_dropout = 5
        for k in range(number_dropout):
            output = model(X)
            output2 = output.data.cpu().numpy()
            output2 = np.squeeze(output2[0])

            output2 = np.argmax(output2,0)
            output2 = np.squeeze(output2)
            pred = cv2.imread(cwd + '\\medical_images\\prediction\\' + filenames_im[j], 0)
            out_result = np.zeros(pred.shape)
            out_result[pred != output2] = 1.0
            out_map.append(out_result)

        out_map = np.array(out_map)
        out_map = np.sum(out_map, 0)/number_dropout
        uncertainty_map = np.zeros(out_map.shape)
        uncertainty_map[out_map > 0.2] = 1

        noisy_lb = lbs[j].copy()
        noisy_lb[noisy_lb > 1] = 1
        gt = gts[j].copy()
        gt[gt > 1] =1
        re = np.concatenate((uncertainty_map, out_map), 1) 
        plt.imshow(re)
        plt.show()
        
        if len(np.unique(gt-noisy_lb)) == 1:
            print('clean')
        else:
            print('noisy')
        pred[uncertainty_map == 1] = noisy_lb[uncertainty_map == 1] 
        re = np.concatenate((noisy_lb, pred, gt), 1)
        plt.imshow(re)
        plt.show()
    else:
        pred = lbs[j].copy()
    save_path = cwd + '\\medical_images\\updated_label\\' + filenames_im[j]
    cv2.imwrite(save_path, pred*200)